In [ ]:
import os
import pdfplumber
import pandas as pd
from io import StringIO

data_folder = os.path.abspath('../papers/')
print("data_folder, ", data_folder)
pdf_path = os.path.join(data_folder, 'ASO_rna_seq_EFO21/rna_seq.pdf')

capture = False

table_to_lines = dict()
tables = range(3, 12)

with pdfplumber.open(pdf_path) as pdf:
    for table_num in tables:
        table_lines = []
        for page in pdf.pages:
            text = page.extract_text()
            for line in text.splitlines():
                if capture and f"Supplementary Table {table_num} lists" in line:
                    capture = False
                    break
                if f"Supplementary Table {table_num}" in line:
                    capture = True
                    continue

                if capture and line.strip():
                    table_lines.append(line.strip())
        table_to_lines[table_num] = table_lines

print(len(table_to_lines))
print(table_to_lines)


In [ ]:
dataframes = []
header = 'ASO ENSEMBL_Transcript_ID ENSEMBL_Gene_ID mean_expression log2(FoldChange) padj'

for table_num, table_line in table_to_lines.items():

    if not table_line[0].startswith('ASO'):
        table_line.insert(0, header)
    else:
        table_line[0] = header

    if table_line[1] == '2':
        table_line.pop(1)

    table_text = '\n'.join(table_line)
    data_io = StringIO(table_text)
    df = pd.read_csv(data_io, delim_whitespace=True)
    dataframes.append(df)

concatenated_df = pd.concat(dataframes, ignore_index=True)

print(concatenated_df)

In [ ]:
paper_path = os.path.join(data_folder, 'ASO_rna_seq_EFO21')
concatenated_df.to_csv(os.path.join(paper_path,'parsed', 'rna_seq.csv'), index=False)